In [3]:
import numpy as np
import tensorflow as tf
import h5py as hpy

tf.__version__

'2.13.0'

In [4]:
train_dataset=hpy.File('train_signs.h5','r')
test_dataset=hpy.File('test_signs.h5','r')

In [5]:
print(test_dataset.keys())
x_train=tf.data.Dataset.from_tensor_slices(train_dataset['train_set_x'])
y_train=tf.data.Dataset.from_tensor_slices(train_dataset['train_set_y'])
x_test=tf.data.Dataset.from_tensor_slices(test_dataset['test_set_x'])
y_test=tf.data.Dataset.from_tensor_slices(test_dataset['test_set_y'])

print(x_test.element_spec)

<KeysViewHDF5 ['list_classes', 'test_set_x', 'test_set_y']>
TensorSpec(shape=(64, 64, 3), dtype=tf.uint8, name=None)


In [6]:
print(next(iter(x_test)))

tf.Tensor(
[[[231 224 216]
  [232 224 216]
  [232 225 217]
  ...
  [226 218 210]
  [226 217 209]
  [225 216 208]]

 [[231 224 215]
  [232 224 215]
  [231 225 216]
  ...
  [226 218 210]
  [225 217 209]
  [224 216 208]]

 [[231 223 215]
  [231 224 215]
  [231 224 216]
  ...
  [225 218 209]
  [225 218 209]
  [224 217 208]]

 ...

 [[201 193 185]
  [201 193 185]
  [201 193 185]
  ...
  [216 204 196]
  [217 204 195]
  [216 204 193]]

 [[201 193 185]
  [201 193 185]
  [201 192 185]
  ...
  [216 204 195]
  [217 204 195]
  [217 204 193]]

 [[200 192 185]
  [200 193 185]
  [200 192 184]
  ...
  [217 204 195]
  [218 204 195]
  [217 204 193]]], shape=(64, 64, 3), dtype=uint8)


In [7]:
unique_labels=set()
for element in y_train:
    unique_labels.add(element.numpy())
print(unique_labels)

{0, 1, 2, 3, 4, 5}


In [8]:
def normalize(image):
    '''
    transform an image into a tensor of shape (64*64*3,) and normalize its components

    '''
    image=tf.cast(image,tf.float32)/255.0
    image=tf.reshape(image,[-1])
    return image

In [9]:
new_train=x_train.map(normalize)
new_test=x_test.map(normalize)

print(new_test.element_spec)

TensorSpec(shape=(12288,), dtype=tf.float32, name=None)


In [10]:
print(next(iter(new_train)))

tf.Tensor([0.8901961  0.8627451  0.8392157  ... 0.8156863  0.81960785 0.81960785], shape=(12288,), dtype=float32)


In [11]:
def linear_function():
    np.random.seed(1)
    
    X=tf.constant(np.random.randn(3,1),name='x')

    W=tf.constant(np.random.randn(4,3),name='W')
    
    b=tf.constant(np.random.randn(4,1),name='b')

    Y=tf.add(tf.matmul(W,X),b)
    
    return Y

print(linear_function())

tf.Tensor(
[[-2.15657382]
 [ 2.95891446]
 [-1.08926781]
 [-0.84538042]], shape=(4, 1), dtype=float64)


In [12]:
def sigmoid(z):

    z=tf.cast(z,tf.float32)
    s=tf.keras.activations.sigmoid(z)

    return s

result=sigmoid(0)
print(result.dtype)
print(result)

<dtype: 'float32'>
tf.Tensor(0.5, shape=(), dtype=float32)


In [13]:
def one_hot_matrix(label,depth=6):
    '''
    label= int, categorical labels
    depth= int, bumber of different classes that label can take
    '''
    one_hot=tf.one_hot(label,depth,axis=0)
    one_hot=tf.reshape(one_hot,[-1])
    return one_hot

In [14]:
new_y_test=y_test.map(one_hot_matrix)
new_y_train=y_train.map(one_hot_matrix)

print(next(iter(new_y_test)))



tf.Tensor([1. 0. 0. 0. 0. 0.], shape=(6,), dtype=float32)


In [15]:
def initialize_parameters():
    '''
    Shapes are:
    W1:[25,12288]
    b1:[25,1]
    W2:[12,25]
    b2:[12,1]
    W3:[6,12]
    b3:[6,1]

    returns parameters
    '''
    initializer=tf.keras.initializers.GlorotNormal(seed=1)

    W1=tf.Variable(initializer(shape=(25,12288)))
    b1=tf.Variable(initializer(shape=(25,1)))
    W2=tf.Variable(initializer(shape=(12,25)))
    b2=tf.Variable(initializer(shape=(12,1)))
    W3=tf.Variable(initializer(shape=(6,12)))
    b3=tf.Variable(initializer(shape=(6,1)))

    parameters={'W1':W1,
                'b1':b1,
                'W2':W2,
                'b2':b2,
                'W3':W3,
                'b3':b3}
    

    return parameters

parameters=initialize_parameters()

    

In [26]:
def forward_propagation(X, parameters):
    '''
    X- input dataset placeholder, of shape (input size, number of examples)

    returns
    Z3-- outpit of the last linear unit
    '''
    W1=parameters['W1']
    b1=parameters['b1']
    W2=parameters['W2']
    b2=parameters['b2']
    W3=parameters['W3']
    b3=parameters['b3']

    Z1=tf.math.add(tf.linalg.matmul(W1,X),b1)
    A1_0=tf.keras.activations.relu(Z1[:,:16])
    A1_1=tf.keras.activations.sigmoid(Z1[:,16:])

    A1_2=tf.concat((A1_0,A1_1),axis=1)
    # print(A1_2.shape)
    
    # A1=tf.keras.activations.relu(Z1)
    Z2=tf.math.add(tf.linalg.matmul(W2,A1_2),b2)
    A2=tf.keras.activations.relu(Z2)
    Z3=tf.math.add(tf.linalg.matmul(W3,A2),b3)

    return Z3


In [30]:
def compute_cost(logits,labels):

    cost=tf.reduce_sum(tf.keras.metrics.categorical_crossentropy(tf.transpose(labels),tf.transpose(logits),from_logits=True))
    return cost

In [18]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
    
    Arguments:
    X_train -- training set, of shape (input size = 12288, number of training examples = 1080)
    Y_train -- test set, of shape (output size = 6, number of training examples = 1080)
    X_test -- training set, of shape (input size = 12288, number of training examples = 120)
    Y_test -- test set, of shape (output size = 6, number of test examples = 120)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 10 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    costs = []                                        # To keep track of the cost
    train_acc = []
    test_acc = []
    
    # Initialize your parameters
    #(1 line)
    parameters = initialize_parameters()

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']

    optimizer = tf.keras.optimizers.Adam(learning_rate)
    
    # The CategoricalAccuracy will track the accuracy for this multiclass problem
    test_accuracy = tf.keras.metrics.CategoricalAccuracy()
    train_accuracy = tf.keras.metrics.CategoricalAccuracy()
    
    dataset = tf.data.Dataset.zip((X_train, Y_train))
    test_dataset = tf.data.Dataset.zip((X_test, Y_test))
    
    # We can get the number of elements of a dataset using the cardinality method
    m = dataset.cardinality().numpy()
    
    minibatches = dataset.batch(minibatch_size).prefetch(8)
    test_minibatches = test_dataset.batch(minibatch_size).prefetch(8)
    #X_train = X_train.batch(minibatch_size, drop_remainder=True).prefetch(8)# <<< extra step    
    #Y_train = Y_train.batch(minibatch_size, drop_remainder=True).prefetch(8) # loads memory faster 

    # Do the training loop
    for epoch in range(num_epochs):

        epoch_cost = 0.
        
        #We need to reset object to start measuring from 0 the accuracy each epoch
        train_accuracy.reset_states()
        
        for (minibatch_X, minibatch_Y) in minibatches:
            
            with tf.GradientTape() as tape:
                # 1. predict
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)

                # 2. loss
                minibatch_cost = compute_cost(Z3, tf.transpose(minibatch_Y))

            # We acumulate the accuracy of all the batches
            train_accuracy.update_state(tf.transpose(Z3), minibatch_Y)
            
            trainable_variables = [W1, b1, W2, b2, W3, b3]
            grads = tape.gradient(minibatch_cost, trainable_variables)
            optimizer.apply_gradients(zip(grads, trainable_variables))
            epoch_cost += minibatch_cost
        
        # We divide the epoch cost over the number of samples
        epoch_cost /= m

        # Print the cost every 10 epochs
        if print_cost == True and epoch % 10 == 0:
            print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            print("Train accuracy:", train_accuracy.result())
            
            # We evaluate the test set every 10 epochs to avoid computational overhead
            for (minibatch_X, minibatch_Y) in test_minibatches:
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)
                test_accuracy.update_state(tf.transpose(Z3), minibatch_Y)
            print("Test_accuracy:", test_accuracy.result())

            costs.append(epoch_cost)
            train_acc.append(train_accuracy.result())
            test_acc.append(test_accuracy.result())
            test_accuracy.reset_states()


    return parameters, costs, train_acc, test_acc

In [29]:
parameters,costs,train_acc,test_acc=model(new_train,new_y_train,new_test,new_y_test,num_epochs=100)

Cost after epoch 0: 0.057808
Train accuracy: tf.Tensor(0.17685185, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.19166666, shape=(), dtype=float32)
Cost after epoch 10: 0.048614
Train accuracy: tf.Tensor(0.40555555, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.49166667, shape=(), dtype=float32)
Cost after epoch 20: 0.042072
Train accuracy: tf.Tensor(0.5574074, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.55833334, shape=(), dtype=float32)
Cost after epoch 30: 0.038114
Train accuracy: tf.Tensor(0.61944443, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.56666666, shape=(), dtype=float32)
Cost after epoch 40: 0.035045
Train accuracy: tf.Tensor(0.6638889, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.575, shape=(), dtype=float32)
Cost after epoch 50: 0.032562
Train accuracy: tf.Tensor(0.68333334, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.6166667, shape=(), dtype=float32)
Cost after epoch 60: 0.030359
Train accuracy: tf.Tensor(0.7287037, shape=(), dt